# Pomiar prędkości

## Projekt ADOM

### Agata Biernacka, Kacper Kilianek

Ze względu na brak możliwości pobrania danych na stronie poradnika wybrano nagranie o podobnym ustawieniu kamery oraz o podobnym oświetleniu by odwzorować warunki autorów.

Niezbędne importy

In [1]:
import cv2
from ultralytics import YOLO, solutions

Inicjalizacja modelu YOLOv8

In [2]:
model = YOLO("yolov8n.pt")
names = model.model.names

Wskazanie źródła nagrania, pobranie rozmiaru klatek i fps

In [3]:
file_path = "./cars_highway.mp4"
cap = cv2.VideoCapture(file_path)
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

Wyznaczenie linii pomocniczej, określenie miejsca zapisu i nazwy filmu z naniesionymi markerami

In [4]:
video_writer = cv2.VideoWriter("cars_highway_analysed.mp4", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))
line_pts = [(0, round(h/2)), (w, round(h/2))]

Inicjalizacja estymatora prędkości

In [5]:
speed_obj = solutions.SpeedEstimator(
    reg_pts=line_pts,
    names=names,
    view_img=True,
)

Analiza klatek nagrania, lokalizacja obiektów, estymacja prędkości oraz zapis markerów

In [6]:
while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    tracks = model.track(im0, persist=True, show=False)

    im0 = speed_obj.estimate_speed(im0, tracks)
    video_writer.write(im0)


0: 384x640 2 cars, 569.4ms
Speed: 3.2ms preprocess, 569.4ms inference, 394.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 5.5ms
Speed: 2.1ms preprocess, 5.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 7.3ms
Speed: 1.6ms preprocess, 7.3ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 8.9ms
Speed: 2.2ms preprocess, 8.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 6.3ms
Speed: 4.1ms preprocess, 6.3ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 6.6ms
Speed: 2.0ms preprocess, 6.6ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 5.5ms
Speed: 2.4ms preprocess, 5.5ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 6.7ms
Speed: 2.1ms preprocess, 6.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars

KeyboardInterrupt: 

Zapis nagrania z markerami

In [ ]:
cap.release()
video_writer.release()
cv2.destroyAllWindows()

### Warunki słabego oświetlenia

In [ ]:
model = YOLO("yolov8n.pt")
names = model.model.names

file_path = "./cars_highway_night.MOV"
cap = cv2.VideoCapture(file_path)
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

video_writer = cv2.VideoWriter("cars_highway_night_analysed.mp4", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))
line_pts = [(0, round(h/2)), (w, round(h/2))]

speed_obj = solutions.SpeedEstimator(
    reg_pts=line_pts,
    names=names,
    view_img=True,
)

while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    tracks = model.track(im0, persist=True, show=False)

    im0 = speed_obj.estimate_speed(im0, tracks)
    video_writer.write(im0)

cap.release()
video_writer.release()
cv2.destroyAllWindows()

### Pomiar prędkości względnej

In [ ]:
import cv2

file_path = "./camera_on_car.MOV"
cap = cv2.VideoCapture(file_path)
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

video_writer = cv2.VideoWriter("camera_on_car.mp4", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))
line_pts = [(0, round(h/2)), (w, round(h/2))]

speed_obj = solutions.SpeedEstimator(
    reg_pts=line_pts,
    names=names,
    view_img=True,
)

# Initialize variables for optical flow
prev_gray = None
prev_keypoints = None

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    # Convert to grayscale for optical flow calculation
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    if prev_gray is None:
        # Initialize the previous frame and keypoints
        prev_gray = gray_frame
        prev_keypoints = cv2.goodFeaturesToTrack(prev_gray, maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)
        continue

    # Calculate optical flow
    curr_keypoints, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, gray_frame, prev_keypoints, None)

    # Select good points
    if curr_keypoints is not None and status is not None:
        good_new = curr_keypoints[status == 1]
        good_old = prev_keypoints[status == 1]

        # Draw the tracks
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()
            c, d = old.ravel()
            frame = cv2.line(frame, (int(a), int(b)), (int(c), int(d)), (0, 255, 0), 2)
            frame = cv2.circle(frame, (int(a), int(b)), 5, (0, 0, 255), -1)

        prev_gray = gray_frame.copy()
        prev_keypoints = good_new.reshape(-1, 1, 2)

        # Track with the model and speed estimation
        tracks = model.track(frame, persist=True, show=False)
        frame = speed_obj.estimate_speed(frame, tracks)
        video_writer.write(frame)
    else:
        print("No valid keypoints found. Reinitializing keypoints.")
        prev_gray = gray_frame
        prev_keypoints = cv2.goodFeaturesToTrack(prev_gray, maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)

cap.release()
video_writer.release()
cv2.destroyAllWindows()
